Importimine

In [6]:
import zipfile
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
from tensorflow.keras import layers, models

Kopsupõletikku tuvastamine

In [7]:

# Path to the ZIP file
zip_path = "archive.zip"
extract_path = "extracted_photos"

# Extract the ZIP file
with zipfile.ZipFile(zip_path, "r") as zip_file:
    zip_file.extractall(extract_path)


zip_path = "archive (1).zip"
# Extract the ZIP file
with zipfile.ZipFile(zip_path, "r") as zip_file:
    zip_file.extractall(extract_path)

zip_path = "archive (2).zip"
# Extract the ZIP file
with zipfile.ZipFile(zip_path, "r") as zip_file:
    zip_file.extractall(extract_path)



In [8]:
import shutil

def create_symlinks(src_dirs, dest_dir):
    if not os.path.exists(dest_dir):
        os.makedirs(dest_dir)

    for src_dir in src_dirs:
        for root, dirs, files in os.walk(src_dir):
            for file in files:
                src_file = os.path.join(root, file)
                relative_path = os.path.relpath(root, src_dir)
                dest_file_dir = os.path.join(dest_dir, relative_path)
                if not os.path.exists(dest_file_dir):
                    os.makedirs(dest_file_dir)
                dest_file = os.path.join(dest_file_dir, file)
                # Copy the file to the destination folder, preserving the directory structure
                if not os.path.exists(dest_file):
                    shutil.copy2(src_file, dest_file)
# Paths to the individual datasets
src_train_dirs = [
    'extracted_photos/chest_xray/chest_xray/train',
    'extracted_photos/Data/train',
    'extracted_photos/Train',
    'extracted_photos/chest_xray/train'
]

src_test_dirs = [
    'extracted_photos/chest_xray/chest_xray/test',
    'extracted_photos/Data/test',
    'extracted_photos/Test',
    'extracted_photos/chest_xray/test'
]

# Path to the combined dataset
combined_train_dir = 'combined_dataset/train'
combined_test_dir = 'combined_dataset/test'

# Create symlinks for train and test data
create_symlinks(src_train_dirs, combined_train_dir)
create_symlinks(src_test_dirs, combined_test_dir)

# ImageDataGenerators for training and testing
train_datagen = ImageDataGenerator(rescale=1.0/255.0)
test_datagen = ImageDataGenerator(rescale=1.0/255.0)

# Create the train and test generators from the combined directory
train_generator = train_datagen.flow_from_directory(
    combined_train_dir,
    target_size=(150, 150),
    batch_size=256,
    class_mode='binary'
)

test_generator = test_datagen.flow_from_directory(
    combined_test_dir,
    target_size=(150, 150),
    batch_size=256,
    class_mode='binary'
)


Found 11125 images belonging to 4 classes.
Found 2070 images belonging to 4 classes.


In [9]:

model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', padding='same', input_shape=(150, 150, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='elu', padding='same'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu', padding='same'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(256, (3, 3), activation='elu', padding='same'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(512, (3, 3), activation='relu', padding='same'),
    layers.Conv2D(1024, (3, 3), activation='elu', padding='same'),
    layers.Conv2D(512, (3, 3), activation='relu', padding='same'),
    layers.Conv2D(256, (3, 3), activation='elu', padding='same'),
    layers.Conv2D(128, (3, 3), activation='relu', padding='same'),
    layers.Flatten(),
    layers.Dense(512, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

model.summary()



history = model.fit(
    train_generator,
    steps_per_epoch=50,  # Number of batches to yield from the generator per epoch
    epochs=30,
    batch_size = 128,
    validation_data=test_generator,
    validation_steps=50
)



test_loss, test_acc = model.evaluate(test_generator, steps=50)
print(f"Test accuracy: {test_acc}")

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_9 (Conv2D)               │ (None, 150, 150, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ (None, 75, 75, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_10 (Conv2D)              │ (None, 75, 75, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_5 (MaxPooling2D)  │ (None, 37, 37, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_11 (Conv2D)              │ (None, 37, 37, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_6 (MaxPooling2D)  │ (None, 18, 18, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_12 (Conv2D)              │ (None, 18, 18, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_7 (MaxPooling2D)  │ (None, 9, 9, 256)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_13 (Conv2D)              │ (None, 9, 9, 512)      │     1,180,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_14 (Conv2D)              │ (None, 9, 9, 1024)     │     4,719,616 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_15 (Conv2D)              │ (None, 9, 9, 512)      │     4,719,104 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_16 (Conv2D)              │ (None, 9, 9, 256)      │     1,179,904 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_17 (Conv2D)              │ (None, 9, 9, 128)      │       295,040 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 10368)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 512)            │     5,308,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │           513 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 17,791,681 (67.87 MB)

 Trainable params: 17,791,681 (67.87 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/30
50/50 ━━━━━━━━━━━━━━━━━━━━ 277s 5s/step - accuracy: 0.2413 - loss: -784006148784128.0000 - val_accuracy: 0.2903 - val_loss: -67293238356082688.0000
Epoch 2/30
50/50 ━━━━━━━━━━━━━━━━━━━━ 271s 5s/step - accuracy: 0.2257 - loss: nan - val_accuracy: 0.0560 - val_loss: nan
Epoch 3/30
50/50 ━━━━━━━━━━━━━━━━━━━━ 229s 4s/step - accuracy: 0.0409 - loss: nan - val_accuracy: 0.0560 - val_loss: nan
Epoch 4/30
50/50 ━━━━━━━━━━━━━━━━━━━━ 217s 4s/step - accuracy: 0.0398 - loss: nan - val_accuracy: 0.0560 - val_loss: nan
Epoch 5/30
50/50 ━━━━━━━━━━━━━━━━━━━━ 219s 4s/step - accuracy: 0.0401 - loss: nan - val_accuracy: 0.0560 - val_loss: nan
Epoch 6/30
50/50 ━━━━━━━━━━━━━━━━━━━━ 218s 4s/step - accuracy: 0.0406 - loss: nan - val_accuracy: 0.0560 - val_loss: nan
Epoch 7/30
50/50 ━━━━━━━━━━━━━━━━━━━━ 239s 4s/step - accuracy: 0.0380 - loss: nan - val_accuracy: 0.0560 - val_loss: nan
Epoch 8/30
50/50 ━━━━━━━━━━━━━━━━━━━━ 232s 4s/step - accuracy: 0.0400 - loss: nan - val_accuracy: 0.0560 - val_loss

KeyboardInterrupt: 

In [ ]:
def load_and_preprocess_image(img_path, target_size=(150, 150)):
    img = image.load_img(img_path, target_size=target_size)
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
    img_array /= 255.0  # Rescale to 0-1 range
    return img_array